# Chat with your code! </>


<img src="chat_with_code.png" width=800px>

In [1]:
import os

os.environ["HF_HOME"] = "/teamspace/studios/this_studio/weights"
os.environ["TORCH_HOME"] = "/teamspace/studios/this_studio/weights"

import gc
import re
import uuid
import textwrap
import subprocess
import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown, display

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.core import PromptTemplate
from llama_index.core import SimpleDirectoryReader
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core import VectorStoreIndex
from llama_index.core.storage.storage_context import StorageContext

from langchain.embeddings import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

from rag_101.retriever import (
    load_embedding_model,
    load_reranker_model
)

In [2]:
# allows nested access to the event loop
nest_asyncio.apply()

In [3]:
# setting up the llm
llm=Ollama(model="llama3", request_timeout=60.0)

# setting up the embedding model
lc_embedding_model = load_embedding_model()
embed_model = LangchainEmbedding(lc_embedding_model)

In [4]:
# utility functions
def parse_github_url(url):
    pattern = r"https://github\.com/([^/]+)/([^/]+)"
    match = re.match(pattern, url)
    return match.groups() if match else (None, None)

def clone_github_repo(repo_url):    
    try:
        print('Cloning the repo ...')
        result = subprocess.run(["git", "clone", repo_url], check=True, text=True, capture_output=True)
    except subprocess.CalledProcessError as e:
        print(f"Failed to clone repository: {e}")
        return None


def validate_owner_repo(owner, repo):
    return bool(owner) and bool(repo)

In [5]:
# Setup a query engine

def setup_query_engine(github_url):
    
    owner, repo = parse_github_url(github_url)
    
    if validate_owner_repo(owner, repo):
        # Clone the GitHub repo & save it in a directory
        input_dir_path = f"/teamspace/studios/this_studio/{repo}"

        if os.path.exists(input_dir_path):
            pass
        else:
            clone_github_repo(github_url)
        
        loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".py", ".ipynb", ".js", ".ts", ".md"],
            recursive=True
        )

        try:
            docs = loader.load_data()

            # ====== Create vector store and upload data ======
            Settings.embed_model = embed_model
            index = VectorStoreIndex.from_documents(docs, show_progress=True)
            # TODO try async index creation for faster emebdding generation & persist it to memory!
            # index = VectorStoreIndex(docs, use_async=True)

            # ====== Setup a query engine ======
            Settings.llm = llm
            query_engine = index.as_query_engine(similarity_top_k=4)
            
            # ====== Customise prompt template ======
            qa_prompt_tmpl_str = (
            "Context information is below.\n"
            "---------------------\n"
            "{context_str}\n"
            "---------------------\n"
            "You are llama3, a large language model developed by Meta AI. Given the context information above I want you to think step by step to answer the query in a crisp manner, incase case you don't know the answer say 'I don't know!'.\n"
            "Query: {query_str}\n"
            "Answer: "
            )
            qa_prompt_tmpl = PromptTemplate(qa_prompt_tmpl_str)

            query_engine.update_prompts(
                {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
            )

            if docs:
                print("Data loaded successfully!!")
                print("Ready to chat!!")
            else:
                print("No data found, check if the repository is not empty!")
            
            return query_engine

        except Exception as e:
            print(f"An error occurred: {e}")
    else:
        print('Invalid github repo, try again!')
        return None

In [6]:
# Provide url to the repository you want to chat with
github_url = "https://github.com/Lightning-AI/lit-gpt"

query_engine = setup_query_engine(github_url=github_url)

Parsing nodes:   0%|          | 0/322 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/531 [00:00<?, ?it/s]

Data loaded successfully!!
Ready to chat!!


In [7]:
response = query_engine.query('Tell me in detail about how to fine tune using lit-gpt?')
display(Markdown(str(response)))

Based on the provided context information, I'll do my best to answer your query.

The repository appears to be related to LitGPT, a collaborative and supportive environment for building with Lightning-Guided Prompt Tuning (Lit-GPT). Here's what I've gathered about the file structure:

* The root directory contains at least two subdirectories: `studios` and `tutorials`.
	+ `studios`: This directory seems to contain examples of LitGPT in action, including a `thunder` subdirectory with a README.md file. This might be an example project or extension for LitGPT.
	+ `tutorials`: This directory contains tutorial materials, including a `0_to_litgpt.md` file that appears to be an introduction to LitGPT. There's also a `resource-tables.md` file with some data and performance metrics related to Lit-GPT.
* The main README.md files (in the root directory and in the `studios` directory) provide information about the project, including instructions on how to contribute and get involved.

I don't have more specific information about the file structure beyond this point. If you're looking for a detailed breakdown of the repository's contents, I recommend exploring the actual files and directories yourself!

Please let me know if there's anything else I can help with!